In [1]:
import pyarrow.parquet as pq
import pandas as pd
import os
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [3]:
os.listdir(os.path.join('.', 'parquets'))

['movies_df', 'movie_title_df', 'netflix_df', 'users_df']

In [4]:
# create dfs from parquet files
netflix_df = pd.read_parquet('parquets/netflix_df')
movie_titles_df = pd.read_parquet('parquets/movie_title_df')
movies_df = pd.read_parquet('parquets/movies_df')
users_df = pd.read_parquet('parquets/users_df')

In [5]:
# merge movies and reviews from movielens dataset
movielens_df = pd.merge(movies_df,users_df,on='movieId',how='outer')

In [6]:
movielens_df

,movieId,imdbId,tmdbId,title,genres,year,userId,rating,tag,timestamp_review,timestamp_tag
0,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0,None,2000-07-30,None
1,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0,None,1996-11-08,None
2,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5,None,2005-01-25,None
3,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5,None,2017-11-13,None
4,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5,None,2011-05-18,None
...,...,...,...,...,...,...,...,...,...,...,...
102879,193581,5476944,432131.0,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017,184,4.0,None,2018-09-16,None
102880,193583,5914996,445030.0,No Game No Life: Zero,Animation|Comedy|Fantasy,2017,184,3.5,None,2018-09-16,None
102881,193585,6397426,479308.0,Flint,Drama,2017,184,3.5,None,2018-09-16,None
102882,193587,8391976,483455.0,Bungo Stray Dogs: Dead Apple,Action|Animation,2018,184,3.5,None,2018-09-16,None


To work around the problem of working with millions of rows, we will append the movie review timestamps and ratings into dicitionaries per movieid. This way, we can work with less rows which will increase the ease of use with the data without taking chunks:

#### Movielens dataset:

In [7]:
# define aggregation functions
agg_funcs = {
    'title': 'first',
    'genres': 'first',
    'year': 'first',
    'userId': lambda x: {idx: val for idx, val in enumerate(x) if pd.notnull(val)} or None,
    'tag': lambda x: {idx: val for idx, val in enumerate(x) if pd.notnull(val)} or None,
    'rating': lambda x: {idx: val for idx, val in enumerate(x) if pd.notnull(val)} or None,
    'timestamp_review': lambda x: {idx: val for idx, val in enumerate(x) if pd.notnull(val)} or None,
    'timestamp_tag': lambda x: {idx: val for idx, val in enumerate(x) if pd.notnull(val)} or None,
    'imdbId': 'first',
    'tmdbId': 'first',
}

# group by 'movieId' and aggregate
movielens_df = movielens_df.groupby('movieId').agg(agg_funcs).reset_index()

In [8]:
# convert genres column to a dictionary
movielens_df['genres'] = movielens_df['genres'].str.split('|').apply(lambda x: {genre: True for genre in x})

In [9]:
movielens_df

,movieId,title,genres,year,userId,tag,rating,timestamp_review,timestamp_tag,imdbId,tmdbId
0,1,Toy Story,"{'Adventure': True, 'Animation': True, 'Childr...",1995,"{0: 1, 1: 5, 2: 7, 3: 15, 4: 17, 5: 18, 6: 19,...","{121: 'pixar', 164: 'pixar', 193: 'fun'}","{0: 4.0, 1: 4.0, 2: 4.5, 3: 2.5, 4: 4.5, 5: 3....","{0: 2000-07-30, 1: 1996-11-08, 2: 2005-01-25, ...","{121: 2006-02-04, 164: 2006-01-14, 193: 2018-0...",114709,862.0
1,2,Jumanji,"{'Adventure': True, 'Children': True, 'Fantasy...",1995,"{0: 6, 1: 8, 2: 18, 3: 19, 4: 20, 5: 21, 6: 27...","{8: 'fantasy', 9: 'magic board game', 10: 'Rob...","{0: 4.0, 1: 4.0, 2: 3.0, 3: 3.0, 4: 3.0, 5: 3....","{0: 1996-10-17, 1: 1996-08-08, 2: 2016-02-16, ...","{8: 2018-06-12, 9: 2018-06-12, 10: 2018-06-12,...",113497,8844.0
2,3,Grumpier Old Men,"{'Comedy': True, 'Romance': True}",1995,"{0: 1, 1: 6, 2: 19, 3: 32, 4: 42, 5: 43, 6: 44...","{26: 'moldy', 27: 'old'}","{0: 4.0, 1: 5.0, 2: 3.0, 3: 3.0, 4: 4.0, 5: 5....","{0: 2000-07-30, 1: 1996-10-17, 2: 2000-08-08, ...","{26: 2006-03-27, 27: 2006-03-27}",113228,15602.0
3,4,Waiting to Exhale,"{'Comedy': True, 'Drama': True, 'Romance': True}",1995,"{0: 6, 1: 14, 2: 84, 3: 162, 4: 262, 5: 411, 6...",None,"{0: 3.0, 1: 3.0, 2: 3.0, 3: 3.0, 4: 1.0, 5: 2....","{0: 1996-10-17, 1: 1996-06-22, 2: 1997-03-19, ...",None,114885,31357.0
4,5,Father of the Bride Part II,{'Comedy': True},1995,"{0: 6, 1: 31, 2: 43, 3: 45, 4: 58, 5: 66, 6: 6...","{37: 'pregnancy', 38: 'remake'}","{0: 5.0, 1: 3.0, 2: 5.0, 3: 3.0, 4: 4.0, 5: 4....","{0: 1996-10-17, 1: 1996-12-13, 2: 1996-11-26, ...","{37: 2006-01-16, 38: 2006-01-16}",113041,11862.0
...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"{'Action': True, 'Animation': True, 'Comedy': ...",2017,{0: 184},None,{0: 4.0},{0: 2018-09-16},None,5476944,432131.0
9738,193583,No Game No Life: Zero,"{'Animation': True, 'Comedy': True, 'Fantasy':...",2017,{0: 184},None,{0: 3.5},{0: 2018-09-16},None,5914996,445030.0
9739,193585,Flint,{'Drama': True},2017,{0: 184},None,{0: 3.5},{0: 2018-09-16},None,6397426,479308.0
9740,193587,Bungo Stray Dogs: Dead Apple,"{'Action': True, 'Animation': True}",2018,{0: 184},None,{0: 3.5},{0: 2018-09-16},None,8391976,483455.0


#### Netflix Prize dataset:

In [10]:
# Extracting arrays directly from DataFrame
user_ids = netflix_df['userId'].values
ratings = netflix_df['rating'].values
dates = netflix_df['date'].values

data = []
current_movie_id = None

# Convert user_ids to a pandas Series to use the .str accessor
user_ids_series = pd.Series(user_ids)

# Find movieIds based on them ending with a colon, strip anything but the last index off, 
# and append the rest of the data in tuples to order everything per movieid
movie_indices = np.where(user_ids_series.str.endswith(':'))[0]
for i, idx in enumerate(movie_indices):
    if i == len(movie_indices) - 1:
        next_idx = len(netflix_df)
    else:
        next_idx = movie_indices[i+1]
    current_movie_id = user_ids[idx].split(':')[0]
    user_data = {idx: val for idx, val in enumerate(netflix_df.loc[idx+1:next_idx-1, 'userId'].tolist())}
    rating_data = {idx: val for idx, val in enumerate(netflix_df.loc[idx+1:next_idx-1, 'rating'].tolist())}
    date_data = {idx: val for idx, val in enumerate(netflix_df.loc[idx+1:next_idx-1, 'date'].tolist())}
    data.append({
        'movieId': current_movie_id,
        'userId': user_data,
        'rating': rating_data,
        'date': date_data
    })

# Create DataFrame from processed data
netflix_df = pd.DataFrame(data, columns=['movieId', 'userId', 'rating', 'date'])

Doing this the id column will be an object, needs to be integer for merging with movie titles:

In [11]:
netflix_df
netflix_df.dtypes

,movieId,userId,rating,date
0,1,"{0: '1488844', 1: '822109', 2: '885013', 3: '3...","{0: 3.0, 1: 5.0, 2: 4.0, 3: 4.0, 4: 3.0, 5: 3....","{0: '2005-09-06', 1: '2005-05-13', 2: '2005-10..."
1,2,"{0: '2059652', 1: '1666394', 2: '1759415', 3: ...","{0: 4.0, 1: 3.0, 2: 4.0, 3: 5.0, 4: 4.0, 5: 2....","{0: '2005-09-05', 1: '2005-04-19', 2: '2005-04..."
2,3,"{0: '1025579', 1: '712664', 2: '1331154', 3: '...","{0: 4.0, 1: 5.0, 2: 4.0, 3: 3.0, 4: 5.0, 5: 4....","{0: '2003-03-29', 1: '2004-02-01', 2: '2004-07..."
3,4,"{0: '1065039', 1: '1544320', 2: '410199', 3: '...","{0: 3.0, 1: 1.0, 2: 5.0, 3: 3.0, 4: 1.0, 5: 1....","{0: '2005-09-06', 1: '2004-06-28', 2: '2004-10..."
4,5,"{0: '1745265', 1: '885013', 2: '1997470', 3: '...","{0: 5.0, 1: 5.0, 2: 5.0, 3: 1.0, 4: 4.0, 5: 5....","{0: '2005-02-08', 1: '2005-05-15', 2: '2005-05..."
...,...,...,...,...
768,13508,"{0: '577397', 1: '44937', 2: '2119144', 3: '25...","{0: 2.0, 1: 1.0, 2: 3.0, 3: 5.0, 4: 1.0, 5: 4....","{0: '2003-12-18', 1: '2003-07-22', 2: '2003-12..."
769,13509,"{0: '998229', 1: '542786', 2: '1977959', 3: '9...","{0: 3.0, 1: 3.0, 2: 3.0, 3: 2.0, 4: 5.0, 5: 5....","{0: '2003-05-29', 1: '2003-07-26', 2: '2002-05..."
770,13510,"{0: '779760', 1: '1296163', 2: '41412', 3: '13...","{0: 3.0, 1: 3.0, 2: 4.0, 3: 4.0, 4: 1.0, 5: 4....","{0: '2005-09-02', 1: '2004-11-01', 2: '2005-01..."
771,13511,"{0: '1392773', 1: '1990901', 2: '2439005', 3: ...","{0: 4.0, 1: 4.0, 2: 4.0, 3: 5.0, 4: 3.0, 5: 2....","{0: '2000-10-01', 1: '2000-05-03', 2: '2004-02..."


movieId    object
userId     object
rating     object
date       object
dtype: object

In [12]:
# convert movieId to integers
netflix_df['movieId'] = netflix_df['movieId'].astype(int)

In [13]:
# merge with movietitles to add title and year
netflix_df = pd.merge(netflix_df,movie_titles_df,on='movieId',how='left')
netflix_df

,movieId,userId,rating,date,year,title
0,1,"{0: '1488844', 1: '822109', 2: '885013', 3: '3...","{0: 3.0, 1: 5.0, 2: 4.0, 3: 4.0, 4: 3.0, 5: 3....","{0: '2005-09-06', 1: '2005-05-13', 2: '2005-10...",2003,Dinosaur Planet
1,2,"{0: '2059652', 1: '1666394', 2: '1759415', 3: ...","{0: 4.0, 1: 3.0, 2: 4.0, 3: 5.0, 4: 4.0, 5: 2....","{0: '2005-09-05', 1: '2005-04-19', 2: '2005-04...",2004,Isle of Man TT 2004 Review
2,3,"{0: '1025579', 1: '712664', 2: '1331154', 3: '...","{0: 4.0, 1: 5.0, 2: 4.0, 3: 3.0, 4: 5.0, 5: 4....","{0: '2003-03-29', 1: '2004-02-01', 2: '2004-07...",1997,Character
3,4,"{0: '1065039', 1: '1544320', 2: '410199', 3: '...","{0: 3.0, 1: 1.0, 2: 5.0, 3: 3.0, 4: 1.0, 5: 1....","{0: '2005-09-06', 1: '2004-06-28', 2: '2004-10...",1994,Paula Abdul's Get Up & Dance
4,5,"{0: '1745265', 1: '885013', 2: '1997470', 3: '...","{0: 5.0, 1: 5.0, 2: 5.0, 3: 1.0, 4: 4.0, 5: 5....","{0: '2005-02-08', 1: '2005-05-15', 2: '2005-05...",2004,The Rise and Fall of ECW
...,...,...,...,...,...,...
768,13508,"{0: '577397', 1: '44937', 2: '2119144', 3: '25...","{0: 2.0, 1: 1.0, 2: 3.0, 3: 5.0, 4: 1.0, 5: 4....","{0: '2003-12-18', 1: '2003-07-22', 2: '2003-12...",1999,The League of Gentlemen: Series 1
769,13509,"{0: '998229', 1: '542786', 2: '1977959', 3: '9...","{0: 3.0, 1: 3.0, 2: 3.0, 3: 2.0, 4: 5.0, 5: 5....","{0: '2003-05-29', 1: '2003-07-26', 2: '2002-05...",1998,Little City
770,13510,"{0: '779760', 1: '1296163', 2: '41412', 3: '13...","{0: 3.0, 1: 3.0, 2: 4.0, 3: 4.0, 4: 1.0, 5: 4....","{0: '2005-09-02', 1: '2004-11-01', 2: '2005-01...",1959,Last Train from Gun Hill
771,13511,"{0: '1392773', 1: '1990901', 2: '2439005', 3: ...","{0: 4.0, 1: 4.0, 2: 4.0, 3: 5.0, 4: 3.0, 5: 2....","{0: '2000-10-01', 1: '2000-05-03', 2: '2004-02...",1993,Much Ado About Nothing


## Joining Netflix and Movielens data:

Now, I will outer join Netflix and Movielens by movieId, year and title. Ratings and timestamps for reviews will be appended in each list:

In [14]:
netflix_df.columns
movielens_df.columns
movielens_df = movielens_df.rename(columns={'timestamp_review': 'date'})

Index(['movieId', 'userId', 'rating', 'date', 'year', 'title'], dtype='object')

Index(['movieId', 'title', 'genres', 'year', 'userId', 'tag', 'rating',
       'timestamp_review', 'timestamp_tag', 'imdbId', 'tmdbId'],
      dtype='object')

The netflix and movielens dataframe have the same dynamics in terms of columns now, including some extras about scores, tags and timestamp of the given tag. Both dfs will be appended and then grouped by movieId, where the userId, rating and date lists will be appended with the new results from the movielens dataframe:

In [15]:
netflix_movielens_df = pd.concat([netflix_df,movielens_df],ignore_index=True)

# show to check afterwards if the indices are not replaced after concatenated dictionaries
netflix_movielens_df[netflix_movielens_df['movieId']==1]

,movieId,userId,rating,date,year,title,genres,tag,timestamp_tag,imdbId,tmdbId
0,1,"{0: '1488844', 1: '822109', 2: '885013', 3: '3...","{0: 3.0, 1: 5.0, 2: 4.0, 3: 4.0, 4: 3.0, 5: 3....","{0: '2005-09-06', 1: '2005-05-13', 2: '2005-10...",2003,Dinosaur Planet,NaN,NaN,NaN,NaN,NaN
773,1,"{0: 1, 1: 5, 2: 7, 3: 15, 4: 17, 5: 18, 6: 19,...","{0: 4.0, 1: 4.0, 2: 4.5, 3: 2.5, 4: 4.5, 5: 3....","{0: 2000-07-30, 1: 1996-11-08, 2: 2005-01-25, ...",1995,Toy Story,"{'Adventure': True, 'Animation': True, 'Childr...","{121: 'pixar', 164: 'pixar', 193: 'fun'}","{121: 2006-02-04, 164: 2006-01-14, 193: 2018-0...",114709.0,862.0


In [16]:
def agg_concat_dicts_np(series):
    # Convert series of dictionaries to a list of dictionaries
    list_of_dicts = list(series)
    # Merge dictionaries
    merged_dict = {}
    for idx, d in enumerate(list_of_dicts):
        if d is not None:
            for key, value in d.items():
                new_key = f'{key}_{idx}'  # Create new key based on original key and index
                merged_dict[new_key] = value
    return merged_dict

# group by 'movieId' and aggregate using the custom function
recsys_df = netflix_movielens_df.groupby('movieId').agg({
    'title' : 'first',
    'year' : 'first',
    'genres' : 'first',
    'imdbId' : 'max',
    'tmdbId' : 'max',
    'userId': agg_concat_dicts_np,
    'rating': agg_concat_dicts_np,
    'date': agg_concat_dicts_np,
    'tag' : 'first',
    'timestamp_tag' : 'first'
}).reset_index()

In [17]:
recsys_df.isnull().sum()
recsys_df

movieId             0
title               0
year               13
genres            477
imdbId            477
tmdbId            485
userId              0
rating              0
date                0
tag              8647
timestamp_tag    8647
dtype: int64

,movieId,title,year,genres,imdbId,tmdbId,userId,rating,date,tag,timestamp_tag
0,1,Dinosaur Planet,2003,"{'Adventure': True, 'Animation': True, 'Childr...",114709.0,862.0,"{'0_0': '1488844', '1_0': '822109', '2_0': '88...","{'0_0': 3.0, '1_0': 5.0, '2_0': 4.0, '3_0': 4....","{'0_0': '2005-09-06', '1_0': '2005-05-13', '2_...","{121: 'pixar', 164: 'pixar', 193: 'fun'}","{121: 2006-02-04, 164: 2006-01-14, 193: 2018-0..."
1,2,Isle of Man TT 2004 Review,2004,"{'Adventure': True, 'Children': True, 'Fantasy...",113497.0,8844.0,"{'0_0': '2059652', '1_0': '1666394', '2_0': '1...","{'0_0': 4.0, '1_0': 3.0, '2_0': 4.0, '3_0': 5....","{'0_0': '2005-09-05', '1_0': '2005-04-19', '2_...","{8: 'fantasy', 9: 'magic board game', 10: 'Rob...","{8: 2018-06-12, 9: 2018-06-12, 10: 2018-06-12,..."
2,3,Character,1997,"{'Comedy': True, 'Romance': True}",113228.0,15602.0,"{'0_0': '1025579', '1_0': '712664', '2_0': '13...","{'0_0': 4.0, '1_0': 5.0, '2_0': 4.0, '3_0': 3....","{'0_0': '2003-03-29', '1_0': '2004-02-01', '2_...","{26: 'moldy', 27: 'old'}","{26: 2006-03-27, 27: 2006-03-27}"
3,4,Paula Abdul's Get Up & Dance,1994,"{'Comedy': True, 'Drama': True, 'Romance': True}",114885.0,31357.0,"{'0_0': '1065039', '1_0': '1544320', '2_0': '4...","{'0_0': 3.0, '1_0': 1.0, '2_0': 5.0, '3_0': 3....","{'0_0': '2005-09-06', '1_0': '2004-06-28', '2_...",None,None
4,5,The Rise and Fall of ECW,2004,{'Comedy': True},113041.0,11862.0,"{'0_0': '1745265', '1_0': '885013', '2_0': '19...","{'0_0': 5.0, '1_0': 5.0, '2_0': 5.0, '3_0': 1....","{'0_0': '2005-02-08', '1_0': '2005-05-15', '2_...","{37: 'pregnancy', 38: 'remake'}","{37: 2006-01-16, 38: 2006-01-16}"
...,...,...,...,...,...,...,...,...,...,...,...
10214,193581,Black Butler: Book of the Atlantic,2017,"{'Action': True, 'Animation': True, 'Comedy': ...",5476944.0,432131.0,{'0_0': 184},{'0_0': 4.0},{'0_0': 2018-09-16},None,None
10215,193583,No Game No Life: Zero,2017,"{'Animation': True, 'Comedy': True, 'Fantasy':...",5914996.0,445030.0,{'0_0': 184},{'0_0': 3.5},{'0_0': 2018-09-16},None,None
10216,193585,Flint,2017,{'Drama': True},6397426.0,479308.0,{'0_0': 184},{'0_0': 3.5},{'0_0': 2018-09-16},None,None
10217,193587,Bungo Stray Dogs: Dead Apple,2018,"{'Action': True, 'Animation': True}",8391976.0,483455.0,{'0_0': 184},{'0_0': 3.5},{'0_0': 2018-09-16},None,None
